In [1]:
import pandas as pd
import requests
import time
import re
from urllib.parse import urlparse
from bs4 import BeautifulSoup as bs
import re
import urllib
from fake_useragent import UserAgent
ua = UserAgent()
headers = { 'User-Agent': ua.chrome}
user_agent_desktop = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '\
'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 '\
'Safari/537.36'
url="https://www.behance.net/search/users?sort=published_date&field=computer%20animation&country=DE"
#response = requests.get(url,headers)

In [2]:
def initialSetup(browser):
    i=0
    error_count=0
    try:
        height = browser.execute_script("return document.body.scrollHeight")
        while i<height:
            browser.execute_script("window.scrollBy(0,150)")
            i+=150
            time.sleep(0.2)
    except:
        error_count+=1

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = Options()
#options=options
options.headless = True
browser = webdriver.Chrome(executable_path="C:/Users/amirr/Documents/Projects/chromedriver")
browser.get(url)
error_count=0
browser.maximize_window()
time.sleep(1)
initialSetup(browser)
time.sleep(2)
initialSetup(browser)
time.sleep(2)
initialSetup(browser)
time.sleep(1)
initialSetup(browser)
time.sleep(1)

In [4]:
profileLink=[]
names=[]
basicStats=[]
workexp=[]
usernames=[]
otherData=[]
finalUrl=[]
soup = bs(browser.page_source, 'lxml')
browser.quit()
profiles=soup.find('ul',{'class':'ContentList-root-39W'})

In [5]:
listItems=profiles.find_all('li',{'class','ContentList-listItem-3sW qa-list-item'})
for item in listItems:
    try:
        mainDiv=item.find('div').find('div')
        profileURL=mainDiv.find('a').get('href')
        profileLink.append(profileURL)
    except:
        print("No Url")
    try:
        mainDiv=item.find('div').find('div')
        profileName=mainDiv.find('div').find('div').get_text().strip()
        names.append(profileName)
    except:
        print("No Name")
        
    

In [6]:
def scrapWork(profile):
    try:
        individualSkills=[]
        val=profile.find_all('li',{'class':'UserInfo-workExperienceEntry-1iy'})
        for exp in val:
            finalVal=""
            company=exp.find_all('div')
            for rowItem in company:
                finalVal+=rowItem.get_text().strip()+" "
            individualSkills.append(finalVal)
        workexp.append(individualSkills)
    except:
        workexp.append('')
    

In [7]:
def scrapName(profile):
    try:
        val=profile.find('h1',{'class':'Profile-userFullName-_EP'}).get_text().strip()
        usernames.append(val)
    except:
        usernames.append("")
    try:
        position=profile.find('div',{'class':'Profile-userDetails-2zj'}).get_text().strip()
        otherData.append(position)
    except:
        otherData.append("")

In [8]:
info=[]
for item in profileLink:
    try:
        response = requests.get(item,headers)
        individualProfile=bs(response.content,'html.parser')
        try:
            res=''
            localStats=[]
            val=individualProfile.find('table',{'class':'UserInfo-userStats-cMw'}).find_all('tr')
            for rowItem in val:
                finalVal=rowItem.get_text().strip()
                res = [re.findall(r'([ \w]+?)([\d,]+)', finalVal)[0] ] 
                localStats.append(res[0])
            basicStats.append(localStats)

        except:
            basicStats.append('')
        try:
            about=individualProfile.find('div',{'class':'UserInfo-bio-YNh'}).get_text().strip()
            about=" ".join(about.split())
            info.append(about)
        except:
            info.append("")
        scrapWork(individualProfile)
        scrapName(individualProfile)
        finalUrl.append(item)
    except:
        print("invalid url")

In [9]:
columns=["Name","Position","Summary","Experience","Statistics"]
df=pd.DataFrame(columns=columns)
df["Name"]=usernames
df["Position"]=otherData
df["Summary"]=info
df["Experience"]=workexp
df["Statistics"]=basicStats
df["ProfileURL"]=finalUrl

In [10]:
import time
ts = time.time()
import datetime
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
st=str(st)

In [271]:
df.to_excel(st+'behance.xlsx', index=False)

In [254]:
usernames

['Michael Schack',
 'Tobias Rosenberg',
 'Vincent Schwenk',
 'Franklin Ponceoyola',
 'María Guðjohnsen',
 'Sven Hauth',
 'Silas Jeydo',
 'Jörg Bandmann',
 'Raphael Rau',
 'Julian Föst',
 'David Baertz',
 'Noi / Amedeo Inglese',
 'David Bernau',
 'Paul Klingberg',
 'Kasem M. Raad',
 'Matteo Forghieri',
 'Manuel Paul',
 'Albert Bauer',
 'Lars Nagler',
 'Chris Smallfield',
 'Oliver Goszyk',
 'Hanna H.',
 'Marco Bergstein',
 'Felix Herbort',
 'Studio 2038',
 'Eugen Fetsch',
 'Denis Braun',
 'Lilli-Luisa Heckmann',
 'Dorian Agu',
 'Aix sponza',
 'qula / Sami',
 'Rene Häfferer',
 'Daria Togunova',
 'Peter Kaletsch',
 'Tobias Eckerlin',
 'Corinna Lippold',
 'Andi Wenzel',
 'Miguel Faber',
 'CG MERSION',
 'Oliver Harries',
 'Yulia Kryachkova',
 'Florian Hofmann',
 'Nikita Christell',
 'Uli Staiger',
 'Jakob Leue',
 'Simon Meier',
 'Hares Parvanta',
 'Claus Winter',
 'Michael Tan',
 'BACKEND Medienservices',
 'RETRIC DREAMS | Maximilian Auer',
 'Franziska Werner',
 'Jan-Christian -Janosch- Grot

In [255]:
otherData

['Medien Gestalter - 3D ArtistSchack-Medien https://www.schack-medien.de/ Lauf, Germany',
 'https://www.tobiasrosenberg.com/ Berlin, Germany',
 'Designer vincentschwenk.de Hamburg, Germany',
 '3D ArtistFreelancer https://www.3dartviz.com Berlin, Germany',
 'Digital Artist mariagudjohnsen.com Berlin, Germany',
 'CG ArtistFreelancer www.pixeldoggy.com Kiel, Germany',
 'Hamburg, Germany',
 'Creative-Director, Motion-Designerpixelwg www.pixelwg.de Leipzig, Germany',
 '3D-Artist, Hyper-RealistSilverwing-VFX www.silverwing-vfx.de Ludwigsburg, Germany',
 'Motion Design | 3D julianfoest.com Augsburg, Germany',
 'Art direction, Concept, Motion. www.davidbaertz.de Herne, Germany',
 'Sound designer - Music ComposerBerlin, Germany',
 'Senior Motion Designer, 3D Generalist www.davidbernau.de Berlin, Germany',
 'Digital ArtistEffekt-EtageBerlin, Germany',
 'ProducerK Productions https://www.kproductions.org/ Berlin, Germany',
 '3d artistBOONDOX lab https://vimeo.com/matteoforghieri Hamburg, Germany'

In [11]:
len(profileLink)

0